In [4]:
import subprocess
import json
import ipywidgets as widgets

def toCmd(strlist):
    return ''.join(str(e + ' ') for e in strlist)[:-1]

##############################################
# different ways  to execute commands
 
# in Kubernetes
def executeKubectlCmd(cmd, decode='ascii'):
    try:    
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl'] + cmd)
        if decode == 'json':
            return json.loads(output.decode('ascii'))
        else:
            return output.decode(decode)
    except Exception as e:
        return str(e)       
     
# at the peer    

def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        
        #print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
       
        return output.decode('ascii')
    except Exception as e:
        return str(e)    
    
def executePeerCmd2(pod_name,cmd):
    try:  
        cmd1 = "source /root/.bashrc && "
        output  = subprocess.run(['/home/jovyan/work/usr/bin/kubectl', 'exec','-it', pod_name, '-c', 'scray-peer-cli','--', '/bin/bash','-c',cmd1 + cmd], stdout = subprocess.PIPE, stderr=subprocess.PIPE)
        return output
    except Exception as e:
        return str(e)    
    
# in Jupyter     
def executeLocalCmd(cmd):
    try:  
        output  = subprocess.check_output(cmd)
        return output
    except Exception as e:
        return str(e)    

def executeLocalCmd2(cmd):
    try:  
        output  = subprocess.run(cmd, stdout = subprocess.PIPE, stderr=subprocess.PIPE)
        return output
    except Exception as e:
        return str(e)        
    

    
##############################################
## peer

def getPort(peername='', name='peer-gossip'):
    callopt= 'jsonpath=\"{.spec.ports[?(@.name==\'' + name + '\')].nodePort}\"'
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','service',peername,'-o', callopt])
        return str(callProcess)[3:-2]
    except Exception as e:
        return str(e)
 
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port='',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=kubernetes.research.dev.seeburger.de',
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=ca_country='  + ca_country,
                                          '--from-literal=ca_province=' + ca_province,
                                          '--from-literal=ca_locality=' + ca_locality,
                                          '--from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)  

#!/home/jovyan/work/usr/bin/kubectl get configmap -o json     
def getConfigmap(): 
    return json.loads(executeKubectlCmd(['get','configmap', '-o', 'json']).decode('ascii'))
       
def getPod(peername):
    out2 = widgets.Output()
    with out2:
        try:    
            callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','pod','-l','app=' + peername,'-o', 
                                                    'jsonpath=\"{.items[0].metadata.name}\"'])
            return callProcess.decode('ascii')[1:-1]
            #return str(callProcess)[3:-2]
        except Exception as e:
            print('exit')
            #return str(e) 
            return None
    
def getPods():
    return json.loads(executeKubectlCmd(['get', 'pods', '-o', 'json']))       